After having installed the openpyxl package:

<pre>
pip install openpyxl
or
conda install openpyxl
</pre>

we are able to create a very simple spreadsheet with the following code:

In [1]:
from openpyxl import Workbook

workbook = Workbook()
sheet = workbook.active

sheet["A1"] = "hello"
sheet["B1"] = "world!"

workbook.save(filename="hello_world.xlsx")

The code above should create a file called hello_world.xlsx in the folder you are using to run the code.

## Reading Excel Spreadsheets With openpyxl

The following examples will use a sample dataset: _sample.xlsx_:

<img src="sales.png" width=800 height=500 />

First you need, to open the spreadsheet sample.xlsx using __load_workbook()__, and then you can use __workbook.sheetnames__ to see all the sheets you have available to work with. 

After that, __workbook.active__ selects the first available sheet and, in this case, you can see that it selects _Sheet 1_ automatically. 

__Note__: There are a few arguments you can pass to __load_workbook()__ that change the way a spreadsheet is loaded. The most important ones are the following two Booleans:

   __read_only__ loads a spreadsheet in read-only mode allowing you to open very large Excel files.<br>
   __data_only__ ignores loading formulas and instead loads only the resulting values.

In [19]:
from openpyxl import load_workbook

workbook = load_workbook(filename="sample.xlsx")
workbook.sheetnames
sheet = workbook.active
sheet
sheet.title

# Check the used spreadsheet space using the attribute "dimensions"
sheet.dimensions

['Sheet1', 'Other', 'Last']

<Worksheet "Sheet1">

'Sheet1'

'A1:P701'

### Reading Cell Values

After opening a spreadsheet, you can easily retrieve data from it's Cells using the __`[]`__ operator.

To return the actual value of a cell, you need to use __.value__. 

You can also use the method __.cell()__ to retrieve a cell using index notation.

__Note__: the first row and the first column are at position 1.

In [12]:
sheet["B5"]

sheet["B5"].value
sheet["E4"].value

sheet.cell(row=9, column=2).value
sheet.cell(10, 2).value

<Cell 'Sheet1'.B5>

'Germany'

2178

'Canada'

'France'

## Iterating Through the Data

There are different ways you can iterate through the data depending on your needs.

You can slice the data with a combination of columns and rows:

In [20]:
# Get a range (a tuple of tuple) of cells 
select1=sheet["A2:C4"]
for a,b,c in select1:
    print(a.value, b.value, c.value)

# Get all cells (a tuple) from column A
sheet["A"]

# Get all cells (a tuple of tuple) for a range of columns
sheet["A:B"]

# Get all cells (a tuple) from row 5
sheet[5]

# Get all cells (a tuple of tuple) for a range of rows
sheet[5:6]


Government Canada Carretera
Government Germany Carretera
Midmarket France Carretera


ValueError: A2:A3,B4:B5 is not a valid coordinate or range

You also go through the data via the methods:

__iter_rows()__<br>
__iter_cols()__<br>

Both methods can receive the following arguments:

__min_row__, __max_row__, __min_col__, __max_col__

These arguments are used to set boundaries for the iteration.

One additional argument you can pass to both methods is the Boolean __values_only__. When it’s set to True, the values of the cell are returned, instead of the Cell object.

__Note__: the 2 attributes __max_row__ and __max_column__ do contain the maximum row/column index containing data.


In [52]:


ix=1
for row in sheet.iter_rows(min_row=2,
                           max_row=3,
                           min_col=1,
                           max_col=3):
    print(f"Row {ix}:", sep=" ")
    for c in row:
        print(f"{c.coordinate} is {c.value}", sep=" ")
    print()
    ix+=1
    
ix=1
for col in sheet.iter_cols(min_row=2,
                           max_row=3,
                           min_col=1,
                           max_col=3,
                           values_only=True):
    print(f"Col {ix}:", sep=" ")
    for c in col:
        print(c, sep=" ")
    print()
    ix+=1

701 16
Row 1:
A2 is Government
B2 is Canada
C2 is Carretera

Row 2:
A3 is Government
B3 is Germany
C3 is Carretera

Col 1:
Government
Government

Col 2:
Canada
Germany

Col 3:
Carretera
Carretera



If you want to iterate through the whole dataset, then you can also use the attributes __rows__ or __columns__ directly,

In [50]:
from operator import attrgetter

for arow in sheet.rows:
    for v in map(attrgetter('value'), arow):
        print (v, end= " ")
    if arow[0].row==5: 
        break
    print()
    
print()

for acol in sheet.columns:
    for v in (c.value for c in acol[:4]):
        print (v, end= " ")
   
    print()

Segment Country Product Discount Band Units Sold Manufacturing Price Sale Price Gross Sales Discounts  Sales COGS Profit Date Month Number Month Name Year 
Government Canada Carretera None 1618.5 3 20 32370 0 32370 16185 16185 2014-01-01 00:00:00 1 January 2014 
Government Germany Carretera None 1321 3 20 26420 0 26420 13210 13210 2014-01-01 00:00:00 1 January 2014 
Midmarket France Carretera None 2178 3 15 32670 0 32670 21780 10890 2014-06-01 00:00:00 6 June 2014 
Midmarket Germany Carretera None 888 3 15 13320 0 13320 8880 4440 2014-06-01 00:00:00 6 June 2014 
Segment Government Government Midmarket 
Country Canada Germany France 
Product Carretera Carretera Carretera 
Discount Band None None None 
Units Sold 1618.5 1321 2178 
Manufacturing Price 3 3 3 
Sale Price 20 20 15 
Gross Sales 32370 26420 32670 
Discounts 0 0 0 
 Sales 32370 26420 32670 
COGS 16185 13210 21780 
Profit 16185 13210 10890 
Date 2014-01-01 00:00:00 2014-01-01 00:00:00 2014-06-01 00:00:00 
Month Number 1 1 6 
Mon

## Adding and Updating Cell Values

You already learned how to add values to a spreadsheet like this:

In [ ]:
sheet["A1"] = "value"

There’s another way you can do this, by first selecting a cell and then changing its value:

In [ ]:
cell = sheet["A1"]
cell.value
cell.value = "hey"
cell.value

__Note__: The new value is only stored into the spreadsheet once you call __workbook.save()__.

__openpyxl__ creates a cell when adding a value, if that cell didn’t exist before:

In [1]:
from openpyxl import load_workbook

workbook = load_workbook(filename=r"C:\Users\jpf\translation2.xlsx")
print(workbook.sheetnames)
sheet=workbook['notification']
languages=[c.value for c in sheet[1][1:]]
sections=workbook.sheetnames
print(sections, type(sections))
dicoLevel1={}
for index, lang in enumerate(languages):
    index += 1
    print(f"lang:{lang}->{index}")
    dicoLevel2={}
    for s in sections:
        print(f"Section: {s}")
        ws=workbook[s]
        dicoLevel3={}
        for r in ws.iter_rows(min_row=2):
            if r[0].value is None:
                break
            dicoLevel3[r[0].value]='' if r[index].value==None else r[index].value
        dicoLevel2[s]=dicoLevel3
    dicoLevel1[lang]=dicoLevel2
    
print(dicoLevel1)
            
            

['notification', 'registration', 'home', 'allEvents', 'cart', 'account', 'event', 'product', 'navigation', 'footer', 'FAQ', 'other']
['notification', 'registration', 'home', 'allEvents', 'cart', 'account', 'event', 'product', 'navigation', 'footer', 'FAQ', 'other'] <class 'list'>
lang:en->1
Section: notification
Section: registration
Section: home
Section: allEvents
Section: cart
Section: account
Section: event
Section: product
Section: navigation
Section: footer
Section: FAQ
Section: other
lang:fr->2
Section: notification
Section: registration
Section: home
Section: allEvents
Section: cart
Section: account
Section: event
Section: product
Section: navigation
Section: footer
Section: FAQ
Section: other
lang:ja->3
Section: notification
Section: registration
Section: home
Section: allEvents
Section: cart
Section: account
Section: event
Section: product
Section: navigation
Section: footer
Section: FAQ
Section: other
{'en': {'notification': {'unableLang': "We were unable to retrieve the lis

### Managing Rows and Columns

One of the most common things you have to do when manipulating spreadsheets is *adding* or *removing* rows and columns. 

The openpyxl package allows you to do that in a very straightforward way by using the methods:

    insert_rows()
    delete_rows()
    insert_cols()
    delete_cols()
    
These methods can receive two arguments:

    1. idx: position of the insertion/deletion (the insertion happens before this value)
    2. amount: number of row/column to insert/delete

In [7]:
import openpyxl as pyxl

workbook = pyxl.Workbook()
sheet = workbook.active

def print_rows(sh):
    for row in sh.iter_rows(values_only=True):
        print(row)
        
sheet["A1"] = "hello"
sheet["B1"] = "world!"

print_rows(sheet)

# Insert a column before the existing column 1 ("A")
sheet.insert_cols(idx=1)
print_rows(sheet)


# Insert 5 columns between column 2 ("B") and 3 ("C")
sheet.insert_cols(idx=3, amount=5)
print_rows(sheet)

# Insert 2 columns before row 1 
sheet.insert_rows(idx=1, amount=2)
print_rows(sheet)

# Delete the created columns
sheet.delete_cols(idx=3, amount=5)
sheet.delete_cols(idx=1)

# Delete the created rows
sheet.delete_rows(idx=1, amount=2)

print_rows(sheet)


('hello', 'world!')
(None, 'hello', 'world!')
(None, 'hello', None, None, None, None, None, 'world!')
(None, None, None, None, None, None, None, None)
(None, None, None, None, None, None, None, None)
(None, 'hello', None, None, None, None, None, 'world!')
('hello', 'world!')


## Managing Sheets

`sheet = workbook.active`

is the way to select the default sheet from a spreadsheet. 

You can select a specific sheet like this:


In [10]:
# To get the sheet titles
workbook.sheetnames

# To select a sheet using its title
sheet1 = workbook["Sheet"]

# To change a sheet title
sheet1.title = "My Data"
workbook.sheetnames


['Sheet']

['My Data']

You can create or delete sheets, using the methods  __`create_sheet()`__ and __`remove()`__:

In [11]:
plots_sheet = workbook.create_sheet("My Plots")
workbook.sheetnames


# You can also define the position to create the sheet at
other_sheet = workbook.create_sheet("Other", 0)
workbook.sheetnames


# To remove them, just pass the sheet as an argument to the .remove()
workbook.remove(other_sheet)
workbook.sheetnames

['My Data', 'My Plots']

['Other', 'My Data', 'My Plots']

['My Data', 'My Plots']

You can duplicate a sheet using __`copy_worksheet()`__

In [12]:
plots_sheet = workbook["My Plots"]
workbook.copy_worksheet(plots_sheet)

workbook.sheetnames

<Worksheet "My Plots Copy">

['My Data', 'My Plots', 'My Plots Copy']

### Freezing Rows and Columns

You might want to freeze a few rows or columns, so they remain visible when you scroll right or down.

Freezing data allows you to keep an eye on important rows or columns, regardless of where you scroll in the spreadsheet.

You can accomplish this by using the worksheet __`freeze_panes`__ attribute:

In [16]:
from openpyxl import load_workbook

workbook = load_workbook(filename="sample.xlsx")
sheet = workbook.active
sheet.freeze_panes = "C3"
workbook.save("sample_frozen.xlsx")

In the `sample_frozen.xlsx` spreadsheet row 1 and row 2 and columns A and B are frozen and are always visible no matter where you navigate within the spreadsheet.

## Adding Formulas

Formulas give us the power to apply specific mathematical equations to a range of cells. 

You can see the list of formulas supported by openpyxl by printing the `dict` __`openpyxl.utils.FORMULAE`__



In [23]:
from openpyxl.utils import FORMULAE
#FORMULAE

from openpyxl import load_workbook

workbook = load_workbook(filename="sample.xlsx")
sheet = workbook.active

sheet["A703"] = "=AVERAGE(E2:E701)"
sheet["A704"] = '=COUNTIF(E2:E701, ">2000")'
workbook.save(filename="sample_formulas.xlsx")

You’ll have to make sure that the strings within a formula are always in double quotes, so you either have to use single quotes around the formula like in the example above or you’ll have to escape the double quotes inside the formula.

### Adding Styles

Using openpyxl, you can apply multiple styling options to your spreadsheet, including fonts, borders, colors, and so on. 

You can choose to either apply a style directly to a cell or create a template and reuse it to apply styles to multiple cells.


In [25]:
# Import necessary style classes
from openpyxl.styles import Font, Color, Alignment, Border, Side

# Create a few styles
bold_font = Font(bold=True)
big_red_text = Font(color="00FF0000", size=20)
center_aligned_text = Alignment(horizontal="center")
double_border_side = Side(border_style="double")
square_border = Border(top=double_border_side,
                       right=double_border_side,
                       bottom=double_border_side,
                       left=double_border_side)

# Style some cells!
sheet["A2"].font = bold_font
sheet["A3"].font = big_red_text
sheet["A4"].alignment = center_aligned_text
sheet["A5"].border = square_border

# Reusing the same styles 
sheet["A6"].alignment = center_aligned_text
sheet["A6"].font = big_red_text
sheet["A6"].border = square_border

workbook.save(filename="sample_styles.xlsx")


When you want to apply multiple styles to one or several cells, you can use the __`NamedStyle`__ class (a kind of style template).

In [36]:
from openpyxl.styles import NamedStyle

# A style template for the header row:
# Create a NamedStyle "header" for the header row (if not already defined)
if 'header' not in workbook.named_styles:
    header = NamedStyle(name="header")
    header.font = Font(bold=True)
    header.border = Border(bottom=Side(border_style="thin"))
    header.alignment = Alignment(horizontal="center", vertical="center")

    # Now let's apply this to all first row (header) cells
    header_row = sheet[1]
    for cell in header_row:
        cell.style = header

workbook.save(filename="sample_styles.xlsx")

### Adding Images

To be able to load images to a spreadsheet using openpyxl, you’ll have to install __`Pillow`__:

`pip install Pillow`

Apart from that, you’ll also need an image and use the method __`add_image()`__. 

In [67]:
from openpyxl import load_workbook
from openpyxl.drawing.image import Image

# Let's use the hello_world spreadsheet 
workbook = load_workbook(filename="hello_world.xlsx")
sheet = workbook.active

logo = Image("epfl.png")

# A bit of resizing 
logo.height = 60
logo.width = 150

sheet.add_image(logo, "C3")
workbook.save(filename="hello_world_logo.xlsx")

### Adding Charts

Charts are a great way to visualize and understand loads of data quickly. There are a lot of different chart types: bar chart, pie chart, line chart, and so on. 

openpyxl has support for a lot (but not all) of them.

openpyxl currently doesn’t have support for are Funnel, Gantt, Pareto, ...

For any chart you want to build, you’ll need to define the chart type: BarChart, LineChart, and so forth, plus the data to be used for the chart, which is called a Reference.


In [68]:
from openpyxl import Workbook
from openpyxl.chart import BarChart, Reference

workbook = Workbook()
sheet = workbook.active

# Let's create some sample sales data
rows = [
    ["Product", "Online", "Store"],
    [1, 30, 45],
    [2, 40, 30],
    [3, 40, 25],
    [4, 50, 30],
    [5, 30, 25],
    [6, 25, 35],
    [7, 20, 40],
]

for row in rows:
    sheet.append(row)

# A bar chart that displays the total number of sales per product:
chart = BarChart()
data = Reference(worksheet=sheet,
                 min_row=1,
                 max_row=8,
                 min_col=2,
                 max_col=3)

chart.add_data(data, titles_from_data=True)
sheet.add_chart(chart, "E2")

workbook.save("chart.xlsx")

In the same way we can quite simply create a  __line chart__.

Here, we are using `from_rows=True` when adding the data. This argument makes the chart plot row by row instead of column by column (in our sample data, each product has a row with 12 values: 1 column per month). 

Here `min_col=1`: the chart expects the first column to have the titles.

There are a couple of other things you can also change regarding the style of the chart. For example, you can add specific categories to the chart:



In [80]:
from openpyxl import Workbook
from openpyxl.chart import LineChart, Reference
import random

workbook = Workbook()
sheet = workbook.active

rows = [
    ["", "January", "February", "March", "April",
    "May", "June", "July", "August", "September",
     "October", "November", "December"],
    [1, ],
    [2, ],
    [3, ],
]

for row in rows:
    sheet.append(row)
for row in sheet.iter_rows(min_row=2,
                           max_row=4,
                           min_col=2,
                           max_col=13):
    for cell in row:
        cell.value = random.randrange(5, 100)
        
chart = LineChart()

data = Reference(worksheet=sheet,
                 min_row=2,
                 max_row=4,
                 min_col=1,
                 max_col=13)

chart.add_data(data, from_rows=True, titles_from_data=True)

cats = Reference(worksheet=sheet,
                 min_row=1,
                 max_row=1,
                 min_col=2,
                 max_col=13)
chart.set_categories(cats)

chart.x_axis.title = "Months"
chart.y_axis.title = "Sales (per unit)"

sheet.add_chart(chart, "C6")

workbook.save("line_chart.xlsx")

### Pandas DataFrame

openpyxl has support for both converting data from a Pandas DataFrame into a workbook or the opposite, converting an openpyxl workbook into a Pandas DataFrame.



In [92]:
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import pandas as pd

data = {
    "Product Name": ["Product 1", "Product 2"],
    "Sales Month 1": [10, 20],
    "Sales Month 2": [5, 35],
}
df = pd.DataFrame(data)
print(df)
workbook = Workbook()
sheet = workbook.active

for row in dataframe_to_rows(df, index=False, header=True):
    sheet.append(row)

workbook.save("pandas.xlsx")

  Product Name  Sales Month 1  Sales Month 2
0    Product 1             10              5
1    Product 2             20             35


PermissionError: [Errno 13] Permission denied: 'pandas.xlsx'

If you want to convert a spreadsheet into a DataFrame, you can also do it in a very straightforward way

In [91]:
import pandas as pd
from openpyxl import load_workbook

workbook = load_workbook(filename="pandas.xlsx")
sheet = workbook.active

data = sheet.values
cols = next(data)
data = list(data)

df = pd.DataFrame(data, columns=cols)

print(df)

  Product Name  Sales Month 1  Sales Month 2
0    Product 1             10              5
1    Product 2             20             35
